# (Dask + XGBoost)

In [1]:
! pip install dask-expr dask xgboost -q


import numpy as np 
import pandas as pd 

import dask

import dask.dataframe as dd

import gc

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.11.0 requires dask==2023.11.0, but you have dask 2024.5.2 which is incompatible.


In [2]:
import os
os.chdir("..")

In [3]:
#po_train= pd.read_csv('Dataset/geolifeclef-2024/GLC24_P0_metadata_train.csv')
pa_train = pd.read_csv('Dataset/geolifeclef-2024/GLC24_PA_metadata_train.csv')
pa_test = pd.read_csv('Dataset/geolifeclef-2024/GLC24_PA_metadata_train.csv')
# sample_sub = pd.read_csv('Dataset/geolifeclef-2024/GLC24_SAMPLE_SUBMISSION.csv')

pa_train = dd.from_pandas(pa_train, npartitions=100)
pa_test = dd.from_pandas(pa_test, npartitions=100)

#print('PO')
#display(po_train.head())
print('PA')
display(pa_train.head())

display(pa_test.head())

gc.collect()

PA


,lon,lat,year,geoUncertaintyInM,areaInM2,region,country,speciesId,surveyId
0,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,6874.0,212
1,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,476.0,212
2,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,11157.0,212
3,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,8784.0,212
4,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,4530.0,212


,lon,lat,year,geoUncertaintyInM,areaInM2,region,country,speciesId,surveyId
0,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,6874.0,212
1,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,476.0,212
2,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,11157.0,212
3,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,8784.0,212
4,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,4530.0,212


116

In [4]:
pa_train['speciesId'] = pa_train['speciesId'].astype(int)
pa_train = pa_train.categorize(columns=['speciesId'])

In [5]:
pa_train.dtypes

lon                          float64
lat                          float64
year                           int64
geoUncertaintyInM            float64
areaInM2                     float64
region               string[pyarrow]
country              string[pyarrow]
speciesId                   category
surveyId                       int64
dtype: object

In [6]:
pa_train.head()

,lon,lat,year,geoUncertaintyInM,areaInM2,region,country,speciesId,surveyId
0,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,6874,212
1,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,476,212
2,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,11157,212
3,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,8784,212
4,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,4530,212


In [7]:
# Excluding rare classes

sum(pa_train.compute()['speciesId'].value_counts() > 40)

1733

In [8]:
used = pa_train.compute()['speciesId'].value_counts() > 40

used = used.iloc[np.where(used)[0]].index.values

len(used)

1733

In [9]:
pa_train.compute().shape

(1483637, 9)

In [10]:
pa_train = pa_train[pa_train['speciesId'].isin(used)]

In [11]:
pa_train.compute().shape

(1450741, 9)

In [12]:
# reset dummies so get_dummies works properly
pa_train['speciesId'] = pa_train.compute()['speciesId'].cat.remove_unused_categories()

In [13]:
ohed_pa_train = dd.get_dummies(pa_train.compute(), columns=['speciesId', ], drop_first=True)

ohed_pa_train.head()

,lon,lat,year,geoUncertaintyInM,areaInM2,region,country,surveyId,speciesId_12,speciesId_30,...,speciesId_11193,speciesId_11194,speciesId_11195,speciesId_11198,speciesId_11215,speciesId_11222,speciesId_11238,speciesId_11240,speciesId_11242,speciesId_11243
0,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,212,False,False,...,False,False,False,False,False,False,False,False,False,False
1,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,212,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,212,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,212,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,212,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
# Grouping the target

In [15]:
grouped_target = ohed_pa_train[ohed_pa_train.columns[7:]]
grouped_target = grouped_target.groupby(by='surveyId').max().reset_index()

pa_features = pa_train[pa_train.columns[:9]].drop_duplicates(subset='surveyId').compute()
pa_features.head()

,lon,lat,year,geoUncertaintyInM,areaInM2,region,country,speciesId,surveyId
1378,9.615880,55.689650,2018,10.0,707.0,CONTINENTAL,Denmark,10684,3547
7836,11.574360,54.730110,2017,10.0,707.0,CONTINENTAL,Denmark,1707,22366
8549,11.686800,55.633510,2019,10.0,79.0,CONTINENTAL,Denmark,8208,24277
9051,3.982406,51.840068,2019,3.0,9.0,ATLANTIC,Netherlands,6491,25783
9080,6.444210,47.695410,2017,1.0,NaN,CONTINENTAL,France,10385,25845


In [16]:
# Cleanup

In [17]:
print(pa_features.isna().sum())

lon                     0
lat                     0
year                    0
geoUncertaintyInM     521
areaInM2             9419
region                  0
country                 0
speciesId               0
surveyId                0
dtype: int64


In [18]:
pa_features['geoUncertaintyInM'] = pa_features['geoUncertaintyInM'].fillna(pa_features['geoUncertaintyInM'].dropna().mean())
pa_features['areaInM2'] = pa_features['areaInM2'].replace([np.inf, -np.inf], np.nan).fillna(pa_features['areaInM2'].replace([np.inf, -np.inf], np.nan).dropna().mean())

In [19]:
pa_features = pa_features.sort_values('surveyId')
grouped_target = grouped_target.sort_values('surveyId')

In [20]:
# One hot encoding of categorical columns
pa_features = dd.get_dummies(pa_features, columns=['region', 'country', 'year'], drop_first=True)

In [21]:
# The result of aggregation:

In [22]:
print('Features')
display(pa_features.head())
print('Multilabel Binary Targets')
display(grouped_target.head())

Features


,lon,lat,geoUncertaintyInM,areaInM2,speciesId,surveyId,region_ATLANTIC,region_BLACK SEA,region_BOREAL,region_CONTINENTAL,...,country_Serbia,country_Slovakia,country_Slovenia,country_Spain,country_Switzerland,country_The former Yugoslav Republic of Macedonia,year_2018,year_2019,year_2020,year_2021
0,3.099038,43.134956,5.0,100.000000,6874,212,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
16,9.884560,56.912140,10.0,79.000000,9816,222,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
21,8.256020,55.637050,10.0,79.000000,5386,243,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
35,-0.402590,43.505630,1.0,123.283588,9028,324,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
41,-0.517360,45.806430,1.0,123.283588,2474,333,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Multilabel Binary Targets


,surveyId,speciesId_12,speciesId_30,speciesId_32,speciesId_35,speciesId_39,speciesId_48,speciesId_51,speciesId_53,speciesId_69,...,speciesId_11193,speciesId_11194,speciesId_11195,speciesId_11198,speciesId_11215,speciesId_11222,speciesId_11238,speciesId_11240,speciesId_11242,speciesId_11243
0,212,False,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
1,222,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,243,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,324,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,333,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
print(pa_features.shape, grouped_target.shape)

(88910, 45) (88910, 1733)


In [24]:
# Base modeling

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    pa_features.drop(columns=['surveyId', 'speciesId']),
    grouped_target.drop(columns='surveyId')
    )


In [26]:
X_train.head()

,lon,lat,geoUncertaintyInM,areaInM2,region_ATLANTIC,region_BLACK SEA,region_BOREAL,region_CONTINENTAL,region_MEDITERRANEAN,region_PANNONIAN,...,country_Serbia,country_Slovakia,country_Slovenia,country_Spain,country_Switzerland,country_The former Yugoslav Republic of Macedonia,year_2018,year_2019,year_2020,year_2021
260286,6.463843,52.515093,3.0,2.25,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
483534,10.260520,56.918170,10.0,707.00,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
1472258,10.354490,57.429560,10.0,79.00,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
1428950,10.621680,55.767060,10.0,79.00,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
537282,9.770890,55.764070,10.0,79.00,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False


In [27]:
X_train.shape, X_val.shape, y_train.shape

((66682, 43), (22228, 43), (66682, 1732))

In [28]:
from xgboost import XGBClassifier


model = XGBClassifier(
    objective='binary:logistic',
    colsample_bylevel=0.90,
    colsample_bytree=0.85,
    colsample_bynode=0.80,
    reg_lambda=0.25,
    reg_alpha=0.70,
    device='gpu',
    early_stopping_rounds=5
    )

model.fit(X_train, y_train,eval_set=[(X_val, y_val)], verbose=5)

[0]	validation_0-logloss:0.11477
[5]	validation_0-logloss:0.04601
[10]	validation_0-logloss:0.03192
[15]	validation_0-logloss:0.02870
[20]	validation_0-logloss:0.02777
[25]	validation_0-logloss:0.02737
[30]	validation_0-logloss:0.02712
[35]	validation_0-logloss:0.02693
[40]	validation_0-logloss:0.02679
[45]	validation_0-logloss:0.02668
[50]	validation_0-logloss:0.02659
[55]	validation_0-logloss:0.02652
[60]	validation_0-logloss:0.02647
[65]	validation_0-logloss:0.02642
[70]	validation_0-logloss:0.02639
[75]	validation_0-logloss:0.02636
[80]	validation_0-logloss:0.02634
[85]	validation_0-logloss:0.02632
[90]	validation_0-logloss:0.02630
[95]	validation_0-logloss:0.02629
[99]	validation_0-logloss:0.02628


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.9, colsample_bynode=0.8,
              colsample_bytree=0.85, device='gpu', early_stopping_rounds=5,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [29]:
model.save_model('models/xgb_clf2')

c:\Users\Jay\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:44:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
